In [10]:
from qiskit import Aer
from qiskit.aqua.algorithms import VQE, ExactEigensolver, QAOA
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua import QuantumInstance
from qiskit.aqua import aqua_globals
from qiskit.circuit.library import RealAmplitudes
from qiskit.visualization import circuit_visualization
from docplex.mp.model import Model
from qiskit.optimization.applications.ising import docplex
# from qiskit import IBMQ
# IBMQ.load_account()
import numpy as np

from collections import OrderedDict

In [11]:
M = [2,6,3]
K = [1,4,2]
T = [12, 6, 24]
d = 19

def get_time_matrix(M, T):
    r = []
    for i in M:
        tmp = []
        for j in T:
            tmp.append(j / i)
        r.append(tmp)
    return np.array(r)


def get_cost_matrix(time_matrix, K):
    m = []
    for i in range(len(time_matrix)):
        tmp = []
        for j in time_matrix[i]:
            tmp.append(K[i] * j)
        m.append(tmp)
    return m

time_matrix = np.array(get_time_matrix(M, T))
cost_matrix = np.array(get_cost_matrix(time_matrix, K))

print("Time matrix:\n {}".format(time_matrix))
print("Cost matrix:\n {}".format(cost_matrix))

Time matrix:
 [[ 6.  3. 12.]
 [ 2.  1.  4.]
 [ 4.  2.  8.]]
Cost matrix:
 [[ 6.  3. 12.]
 [ 8.  4. 16.]
 [ 8.  4. 16.]]


In [12]:
def sample_most_likely(state_vector):
    if isinstance(state_vector, (OrderedDict, dict)):
        # get the binary string with the largest count
        binary_string = sorted(state_vector.items(), key=lambda kv: kv[1])
        repetitions = int(binary_string[-1][1])
        binary_string = binary_string[-1][0]
        x = np.asarray([int(y) for y in reversed(list(binary_string))])
        return x, repetitions
    return [], 0

optimal_key = "0000001000"

def get_stats_for_result(dict_res):
    optimal = 0
    correct = 0
    incorrect = 0
    correct_config = 0
    incorrect_config = 0
     
    if optimal_key in dict_res:
        optimal = dict_res[optimal_key]
    for key, val in dict_res.items():
        if is_correct(key):
            correct += val
            correct_config += 1
        else:
            incorrect += val
            incorrect_config += 1
            
    print('most likely solution: ',sample_most_likely(dict_res))      
    print("optimal: ", optimal)
    print("correct solutions: ", correct)
    print("incorrect solutions: ", incorrect)
    print("correct configs: ", correct_config)
    print("incorrect configs: ", incorrect_config)

def is_correct(key):
    reversed_key = key[::-1]
    if solution_vector_correct(reversed_key) and execution_time(reversed_key) <= d:
        return True
    return False

correct_machines = ['00', '01', '11']
machine_to_index = {'00': 0, '01': 1, '11': 2}

def solution_vector_correct(vector):
    task1_machine = vector[0:2]
    task2_machine = vector[2:4]
    task3_machine = vector[4:6]

    if task1_machine in correct_machines and task2_machine in correct_machines and task3_machine in correct_machines:
        return True
    return False

def execution_time(k):
    task1_machine = machine_to_index.get(k[0:2])
    task2_machine = machine_to_index.get(k[2:4])
    task3_machine = machine_to_index.get(k[4:6])
    
    task1_time = time_matrix[task1_machine, 0]
    task2_time = time_matrix[task2_machine, 1]
    task3_time = time_matrix[task3_machine, 2]
    
    slack_sum = int(k[6]) * 8 + int(k[7]) * 4 + int(k[8]) * 2 + int(k[9]) * 1
    
    return task1_time + task2_time + task3_time + slack_sum

In [13]:
def getMdl(A,B,C):
    mdl = Model(name='workflow')
    x = {(i, j) : mdl.binary_var(name='x_{0}_{1}'.format(i, j)) for i in range(0, 3) for j in range(0, 2)}
    x.update({i : mdl.binary_var(name='x_{0}'.format(i)) for i in range(7, 11)})
    
    objective = A* mdl.sum(cost_matrix[2,i] * x[(i,0)]   
                            + cost_matrix[1,i] * (x[(i,1)] - x[(i,0)]) ** 2
                            + cost_matrix[0,i] * (1 - x[(i,1)]) for i in range(0, 3))
    
    mdl.add_constraint(B * mdl.sum((x[(i,0)] - x[(i,0)] * x[(i,1)]) for i in range(0, 3)) == 0, "machine usage")

    mdl.add_constraint(C * (mdl.sum([time_matrix[2,i] * x[(i,0)]   
                            + time_matrix[1,i] * (x[(i,1)] - x[(i,0)])**2
                            + time_matrix[0,i] * (1 - x[(i,1)]) for i in range(0, 3)]) 
                                  + (8 * x[7] + 4 * x[8] + 2 * x[9] + x[10])) == C * d, "deadline")
    
    mdl.minimize(objective)
    
    print("Number of quadratic constraints: {}".format(mdl.number_of_quadratic_constraints))    
    print(mdl.export_as_lp_string())
    
    H, _ = docplex.get_operator(mdl)
    return H

In [14]:
seed = 123569
aqua_globals.random_seed = seed
entanglement='full'

reps = 2
max_trials = 1000
shots = 100

A = 2
B = 10
C = 7

qubitOp = getMdl(A, B, C)

Number of quadratic constraints: 2
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: workflow

Minimize
 obj: 16 x_0_0 - 12 x_0_1 + 8 x_1_0 - 6 x_1_1 + 32 x_2_0 - 24 x_2_1 + [
      32 x_0_0^2 - 64 x_0_0*x_0_1 + 32 x_0_1^2 + 16 x_1_0^2 - 32 x_1_0*x_1_1
      + 16 x_1_1^2 + 64 x_2_0^2 - 128 x_2_0*x_2_1 + 64 x_2_1^2 ]/2 + 42
Subject To
 machine_usage: [ - 10 x_0_0*x_0_1 - 10 x_1_0*x_1_1 - 10 x_2_0*x_2_1 ]
                + 10 x_0_0 + 10 x_1_0 + 10 x_2_0 = 0
 deadline: [ 14 x_0_0^2 - 28 x_0_0*x_0_1 + 14 x_0_1^2 + 7 x_1_0^2
           - 14 x_1_0*x_1_1 + 7 x_1_1^2 + 28 x_2_0^2 - 56 x_2_0*x_2_1
           + 28 x_2_1^2 ] + 28 x_0_0 - 42 x_0_1 + 14 x_1_0 - 21 x_1_1 + 56 x_2_0
           - 84 x_2_1 + 56 x_7 + 28 x_8 + 14 x_9 + 7 x_10 = -14

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= x_1_0 <= 1
 0 <= x_1_1 <= 1
 0 <= x_2_0 <= 1
 0 <= x_2_1 <= 1
 0 <= x_7 <= 1
 0 <= x_8 <= 1
 0 <= x_9 <= 1
 0 <= x_10 <= 1

Binaries
 x_0_0 x_0_1 x_1_0 x_1_1 x_2_0 x_2_1 x_7 x_8 x_9 

In [15]:
spsa = SPSA(maxiter=max_trials)
ry = RealAmplitudes(qubitOp.num_qubits, reps=reps, entanglement=entanglement)
vqe = VQE(qubitOp, ry, spsa)
# provider= IBMQ.get_provider('ibm-q')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend,seed_simulator=seed, seed_transpiler=seed, shots=shots)
result = vqe.run(quantum_instance)   
state_vector = result['eigenstate']
get_stats_for_result(state_vector)

most likely solution:  (array([1, 1, 0, 1, 0, 0, 0, 0, 1, 0]), 44)
optimal:  0
correct solutions:  50
incorrect solutions:  50
correct configs:  4
incorrect configs:  13


In [16]:
from qiskit.aqua.algorithms import NumPyEigensolver

eigensolver = NumPyEigensolver(qubitOp, 3)
print(eigensolver.compute_eigenvalues())

{'eigenvalues': array([-2083221.5+0.j, -2083219.5+0.j, -2083217.5+0.j]), 'eigenstates': ListOp([VectorStateFn(Statevector([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2)), coeff=1.0, is_measurement=False), VectorStateFn(Statevector([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2)), coeff=1.0, is_measurement=False), VectorStateFn(Statevector([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2)), coeff=1.0, is_measurement=False)], coeff=1.0, abelian=False)}


In [17]:
vqe.get_optimal_cost()
#qaoa.get_optimal_vector()

-1978572.7100000007

In [18]:
vqe.get_optimal_vector()
#qaoa.get_optimal_vector()

{'0100001000': 1,
 '0100001010': 3,
 '0100001011': 44,
 '0100001111': 2,
 '0110001010': 2,
 '0110001011': 23,
 '0110001111': 1,
 '1100001011': 7,
 '0001100000': 2,
 '0001100001': 5,
 '0001101000': 1,
 '0001101001': 1,
 '0010001011': 3,
 '0010001111': 1,
 '0000001011': 2,
 '0011100000': 1,
 '0011101001': 1}